# Preparation

<b>Load Libraries</b>

In [1]:
# bs4
import requests
from bs4 import BeautifulSoup

# selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement

# data structures
import numpy as np
import pandas as pd

# others
import sys, os, re, datetime

<b>UDF</b>

In [2]:
# get scrapping time
def get_scrapping_time() -> str:
    return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [3]:
# get pages to iterate
def get_pages(browser: WebDriver) -> list:
    pages = []
    for button in browser.find_elements(By.XPATH, "//span[@class='ButtonLabel']"):
        try:
            float(button.text.strip())
            pages.append(button)
        except:
            continue

    return pages

In [4]:
# extract table from each page
def extract_data(rows: WebElement, table: dict) -> None:
    keys = list(table.keys)
    for row in rows:
        ## table
        tds = [td.text().strip() for td in enumerate(row.find_elements(By.XPATH, ".//child::td")) if td.text.strip() != '']
        for i, td in enumerate(tds):
            table[keys[i]] = td
        ## homecard
        

In [5]:
# get the button for switching to table format
def get_table_button(browser):
    try:
        ## box
        button = browser.find_element(By.XPATH, "//span[@data-text='Table']")
    except:
        ## drop-down
        button = browser.find_element(By.XPATH, "//select[@role='combobox']//option[@value='table']")

    return button

In [6]:
def login_to_redfin(email: str, password: str, browser: WebDriver) -> None:
    browser.implicitly_wait(10)
    ## login button
    browser.implicitly_wait(10)
    browser.find_element(By.XPATH, "//span[text()='Join / Sign in']").click()
    # email
    browser.find_element(By.XPATH, "//input[@id='emailInput']").send_keys(email)
    browser.find_element(By.XPATH, "//span[text()='Continue with email']").click()
    browser.find_element(By.XPATH, "//span[text()='Sign in with email instead']").click()
    ## password
    browser.find_element(By.XPATH, "//input[@id='passwordInput']").send_keys(password)
    ## finish
    browser.find_element(By.XPATH, "//span[text()='Continue with email']").click()

    return None

# Scraping - Selenium

In [7]:
## options
chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
## browser
browser = webdriver.Chrome(options=chrome_options)
browser.implicitly_wait(10)

In [8]:
# switch to table format
browser.get('https://www.redfin.com/city/29470/IL/Chicago')
button = get_table_button(browser)
button.click()

In [9]:
# login to redfin
# email = 'john.lukestein@gmail.com'
# password = 'redfin.0504'
# login_to_redfin(email, password, browser)

In [10]:
# by using this xpath we'll exclude the first table with no rows
rows_to_click = browser.find_elements(By.XPATH, "//table//tbody[@class='tableList']//tr")
# rows_to_click = table.find_elements(By.XPATH, "//tr")

In [16]:
homecard = browser.find_element(By.XPATH, "//div[@class='TableViewHomecardContent flex flex-column']")

In [27]:
agent_name = homecard.find_element(By.XPATH, "//p[@class='ListingAgentCard__name']").text

In [24]:
titles = homecard.find_elements(By.XPATH, ".//descendant::span[@class='title']")
values = homecard.find_elements(By.XPATH, ".//descendant::span[@class='value']")

In [28]:
browser.quit()